<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

# Streaming Example

### Description:
This script comes to demonstrate working with streaming mode with one channel on a Proteus device.


### set up:
Use Proteues device and connect channel 1 to the scoop.
This scripts works with PXI only because we need connection

### Proteus version: 
ALICE

# Streaming Mode

When working with streaming mode, the main application should continouosly push packets of waveform-data <br>
to instrument. The packets have constant size of 1 Mega ($1024\cdot1024$) bytes of waveform data.<br>

The operation of pushing packets to instrument must be very fast so the main application should run on the same<br>
computer that controls the hardware modules. Further more, the main application should load and run the DLL that<br> controls the hardware modules, and call the function that pushes a packet directly from the DLL.<br>

This means that the main application does not send SCPI commands over TCP to server that runs the hardware<br> controller, but passes the SCPI commands by calling appropriate function from the DLL.

Please note that if there is server running on the computer that controls the hardware, then it should be stop<br>
beacuse when it runs the DLL won't detect the hardware modules.

The DLL is called `TEProteus.dll`. It is a pure `c` DLL rather than *Dot Net* DLL (which might be slower), and<br>
it comes with header file `TEProteus.h` and lib file `TEProteus.lib` so it should be possible to load it from MATLB.<br>

In python one can use `teproteus` that loads the `TEProteus.dll` and gives access to its functions.<br>

In order to work in streaming mode

 1. Select channel by sending SCPI command: `:INSTrument:CHANnel[:SELect] <channel-number>`
 2. Acquire the stream-writing interface of the selected channel by calling the function<br>
    `TEProteusInst.acquire_stream_intf(chan_num)` from `teproteus`. Please note that currently<br>
    only single channel can work in streaming-mode.
 3. Start streaming mode in the selected channel by sending the SCPI command `:TRACe:STReaming ON`.
 4. Push packts of waveform data continouosly either with the function <br>
     - `TEProteusInst.push_stream_packet(self, bin_dat, bytes_offs, usec_wait)`  
     
    or with the lower level functions:<br>
     - `TEProteusInst.get_stream_empty_buff(self)`.
     - `TEProteusInst.put_stream_full_buff(self, full_buff, usec_wait)`.
     - `TEProteusInst.put_stream_empty_buff(self, empty_buff)`.  
    
    See their documentation in `teproteus.py`.
 5. Stop the streaming-mode at the end by sending the SCPI command `:TRACe:STReaming OFF`.


## Simple example
In this very simple example we create 3 waveforms, each of them in size of 64 packets,<br>
and push the packets of the selected waveform (which we change every 30 seconds) in cyclic mode.

import os
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst

In [9]:
%matplotlib notebook
import numpy as np
import time
from threading import Thread
import ipywidgets as widgets
from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt

## Prepare waves

In [10]:
def prepare_waves(inst, dac_mode, packet_size, num_packets):
    
    if dac_mode == 16:
        max_dac = 65535
        data_type = np.uint16 
        packetLen = packet_size // 2
    else:
        max_dac = 255
        data_type = np.uint8 
        packetLen = packet_size
    
    half_dac = max_dac / 2.0

    totalLen = num_packets * packetLen
    totalSize = num_packets * packet_size

    numCycles = 1
    cycleLen = totalLen / numCycles


    x = np.linspace(0, 2 * np.pi * numCycles, totalLen, False)
    w1 = np.sin(x)
    w2 = np.arcsin(w1) * 2 / np.pi
    w3 = (w1 >= 0) * max_dac

    del x

    np.round((w1 + 1) * half_dac, out=w1)
    np.clip(w1, 0, max_dac, out=w1)

    np.round((w2 + 1) * half_dac, out=w2)
    np.clip(w2, 0, max_dac, out=w2)

    if 8 == dac_mode:
        w1 = w1.astype(np.uint8)
        w2 = w2.astype(np.uint8)
        w3 = w3.astype(np.uint8)
    else:
        w1 = w1.astype(np.uint16)
        w2 = w2.astype(np.uint16)
        w3 = w3.astype(np.uint16)

    return w1, w2, w3

## Start Streaming Mode

In [11]:
def start_streaming_mode(inst):
    # Select channel
    rc = inst.send_scpi_cmd(':INST:CHAN 1')

    #Turn output on
    rc = inst.send_scpi_cmd(':OUTP ON')

    # Select sampling rate
    rc = inst.send_scpi_cmd(':FREQ:RAST 3.5e9')

    # Turn streaming mode on

    # Send :TRACe:STReaming ON
    rc = inst.send_scpi_cmd(':TRAC:STR ON')

    time.sleep(0.1)

    resp = inst.send_scpi_query(':SYST:ERR?')
    print(resp)

    resp = inst.send_scpi_query(':TRAC:STR?')
    print('Streaming state: ' + resp)
    

## Push packets of waveform data continouosly

In [12]:
def push_streaming_data(inst, waves, num_packets, packet_size):
           
    iwav = 0
    wav = waves[iwav]

    ipacket = 0
    byteOffs = 0

    # The maximal waiting-time when
    # pushing a full buffer (in microseconds)
    usWait = 50000
    
    global keep_looping

    t1 = time.time()    
    try:
        #is_write_stream_active = inst.is_write_stream_active()
        
        #if not is_write_stream_active:
        #    raise RuntimeError('is_write_stream_active is False')
        
        # This is the stream-pushing loop  ..
        while keep_looping:
            byteOffs = ipacket * packet_size

            # Push next packet (from the selected waveform)
            rc = inst.push_stream_packet(wav, byteOffs, usWait)
            if rc == -1:
                break

            # Change the selected waveform every 30 seconds
            t2 = time.time()
            if t2 - t1 >= 30:            
                t1 = time.time()
                iwav = (iwav + 1) % 3
                wav = waves[iwav]
            ipacket = (ipacket + 1) % num_packets

    except exception:
        pass

    # At the end send ':TRACe:STReaming OFF' to stop streaming mode
    rc = inst.send_scpi_cmd(':TRAC:STR OFF')
    # Just for debug ..
    resp = inst.send_scpi_query(':TRAC:STR?')
    print('Streaming state: ' + resp)


## Main Part

In [14]:
inst = None
keep_looping = True

def main_func():
    
    print('Start of main function')
    
    # Create a single instance of instruments-administrator.
    # Using the 'with' statement makes sure it will be closed at the end
    # Otherwise the user should close the administrator at the end with admin.close_inst_admin()

    with TepAdmin() as admin:

        # Open a single-slot instrument
        
        global inst

        # Please select appropriate slot-id
        inst = admin.open_instrument(slot_id=5)

        resp = inst.send_scpi_query('*IDN?')
        print('Connected to: ' + resp)

        # Optionaly set paranoia-level (by default it is 1).
        # The supported values are: low (0), normal (1), or debug (2).
        inst.default_paranoia_level = 2

        # Reset the instrument
        rc = inst.send_scpi_cmd('*CLS; *RST')

        # Get the model name
        resp = inst.send_scpi_query(":SYST:iNF:MODel?")
        print("Model: " + resp)
        
        # Infer the natural DAC waveform format
        if 'P9082' in resp:
            dac_mode = 8
        else:
            dac_mode = 16
        print("DAC waveform format: {0} bits-per-point".format(dac_mode))

        # Get packet size:
        packet_size = inst.get_stream_packet_size()

        print('Preparing waves ..')

        num_packets = 64

        # Prepare 3 waveforms (each in size of 64 packets)
        waves = prepare_waves(inst, dac_mode, packet_size, num_packets)

        # Acquire the stream-writing interface of the specified channel:
        inst.acquire_stream_intf(chan_num = 1)

        time.sleep(0.1)    

        # Start the streaming mode (in channel 1):
        start_streaming_mode(inst)

        global keep_looping
        
        keep_looping = True
        
        # Push streaming-data while keep_looping is True
        push_streaming_data(inst, waves, num_packets, packet_size)
        
        # Close instrument:
        print('Closing instrument ..')    
        inst.close_instrument()
        
    print('End of main function') 

# Run the main-function in thread
main_thread = Thread(target = main_func)

stop_streaming_button = widgets.Button(description='Stop Streaming')

def on_stop_streaming_button_clicked(arg):
    global inst
    global keep_looping
    global main_thread
    print('stop-streaming button clicked')
    keep_looping = False
    time.sleep(0.2)
    #if inst is not None:
    #    inst.send_scpi_cmd(':TRAC:STR OFF')
    #    inst.close_instrument()
    main_thread.join(5.0)

stop_streaming_button.on_click(on_stop_streaming_button_clicked)


# Start the main thread
main_thread.start()

# Display the button

display(stop_streaming_button)


    
    

Start of main function


Button(description='Stop Streaming', style=ButtonStyle())

Load TEProteus.dll from : C:\WINDOWS\system32\TEProteus.dll
Connected to: Tabor Electronics,P9484M,000002221014,1.223.0
Model: P9484M
DAC waveform format: 16 bits-per-point
Preparing waves ..
0, no error
Streaming state: OFF
Streaming state: OFF
Closing instrument ..
End of main function
